In [1]:
# 🛠️ Etapa 1: Instalações
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers accelerate peft bitsandbytes datasets evaluate bert_score rouge_score nltk sacrebleu fuzzywuzzy[speedup]
!pip install -q tensorboard

# 📂 Imports
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset
import pandas as pd
import matplotlib.pyplot as plt
import json
import os

# 🧭 Caminho do dataset original
dataset_path = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/dados_treinamento_guarani.json"

# ✅ Carregando o dataset original
with open(dataset_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Garante que esteja no formato correto
if isinstance(data, dict):
    data = [data]
elif isinstance(data, str):
    data = [json.loads(l) for l in data.splitlines()]

# ✍️ Função para formatar prompt estilo instruct
def format_prompt(example):
    if example["input"]:
        prompt = f"Instrução: {example['instruction']}\nEntrada: {example['input']}\nResposta:"
    else:
        prompt = f"Instrução: {example['instruction']}\nResposta:"
    return {"prompt": prompt, "output": example["output"]}

# ✅ Criação do Dataset Hugging Face
dataset_original = Dataset.from_list(data)
dataset_original = dataset_original.train_test_split(test_size=0.1, seed=42)
dataset_original = dataset_original.map(format_prompt)

# ✅ Exibe estatísticas
print(f"📊 Dataset original — Treino: {len(dataset_original['train'])} | Teste: {len(dataset_original['test'])}")


Mounted at /content/drive
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6

Map:   0%|          | 0/587 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

📊 Dataset original — Treino: 587 | Teste: 66
